In [17]:
import os
import numpy as np
from sklearn import metrics
from simpletransformers.classification import ClassificationModel
import pandas as pd
from scipy.special import softmax

In [18]:
path_model = '/home/manuto/results_simpletransformers_bert-base-uncased_may5_7Klabels_0/bert-base-uncased_may5_7Klabels_is_hired_1mo/models/checkpoint-111-epoch-1'
path_eval_data = '/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_hired_1mo.csv'

In [19]:
model = ClassificationModel('bert', path_model, args={'evaluate_during_training': True, 'evaluate_during_training_verbose': True, 'num_train_epochs': 20})
print(path_eval_data)
eval_df = pd.read_csv(path_eval_data)
eval_df.columns = ["id", "text", "labels"]
eval_df = eval_df[["text","labels"]]
result, model_outputs, wrong_predictions = model.eval_model(eval_df)
probabilities = np.array([softmax(element) for element in model_outputs])
y_pred = np.array([element[1] for element in probabilities])
eval_df["y_pred_proba"] = y_pred
eval_df.head()

/home/manuto/twitter/data/may5_7Klabels/data_binary_pos_neg_balanced_removed_allzeros/val_is_hired_1mo.csv


HBox(children=(FloatProgress(value=0.0, max=220.0), HTML(value='')))

HBox(children=(FloatProgress(value=0.0, max=28.0), HTML(value='')))

,text,labels,y_pred_proba
0,@jonniker Quinn was self employed and is now l...,0,0.034043
1,whats funny is the job i had today wasnt even ...,1,0.786425
2,I have been working for 4 days but my dependen...,1,0.051963
3,Yesterday I got a job and today I got a call f...,1,0.708000
4,Crazy how I lost my job then got a different p...,1,0.756090


In [20]:
def create_predict_label_from_proba(x):
    if x > 0.5:
        return 1
    else:
        return 0

In [21]:
eval_df["y_pred"] = eval_df["y_pred_proba"].apply(create_predict_label_from_proba)
eval_df.head()

,text,labels,y_pred_proba,y_pred
0,@jonniker Quinn was self employed and is now l...,0,0.034043,0
1,whats funny is the job i had today wasnt even ...,1,0.786425,1
2,I have been working for 4 days but my dependen...,1,0.051963,0
3,Yesterday I got a job and today I got a call f...,1,0.708000,1
4,Crazy how I lost my job then got a different p...,1,0.756090,1


In [23]:
TP = 0
TN = 0
FP = 0
FN = 0
for i in range(eval_df.shape[0]):
    if eval_df["labels"][i] == 1 and eval_df["y_pred"][i] == 1:
        TP = TP + 1
    elif eval_df["labels"][i] == 0 and eval_df["y_pred"][i] == 0:
        TN = TN + 1
    elif eval_df["labels"][i] == 1 and eval_df["y_pred"][i] == 0:
        FN = FN + 1
    elif eval_df["labels"][i] == 0 and eval_df["y_pred"][i] == 1:
        FP = FP + 1
print("*********** Results from test ***********")
print("TP: ", TP)
print("TN: ", TN)
print("FN: ", FN)
print("FP: ", FP)
print('Recall:', str(TP/(TP+FN)),  'Precision', str(TP/(TP+FP)))
print("*********** Output results from simpletransformers ***********")
print(result)

*********** Results from test ***********
TP:  94
TN:  97
FN:  20
FP:  9
Recall: 0.8245614035087719 Precision 0.912621359223301
*********** Output results from simpletransformers ***********
{'mcc': 0.7406677733546588, 'tp': 94, 'tn': 97, 'fp': 9, 'fn': 20, 'eval_loss': 0.38582159232880386}


In [24]:
eval_df = eval_df[["labels","y_pred_proba"]]
eval_df.columns = ['pos','pos_model']
eval_df['neg_model'] = abs(1-eval_df['pos_model'])
eval_df['neg'] = abs(1-eval_df['pos'])
eval_df.head()

,pos,pos_model,neg_model,neg
0,0,0.034043,0.965957,1
1,1,0.786425,0.213575,0
2,1,0.051963,0.948037,0
3,1,0.708000,0.292000,0
4,1,0.756090,0.243910,0


In [26]:
threshold = 0.5
TP = np.sum(eval_df.loc[eval_df['pos_model'] > threshold, 'pos'])
FP = np.sum(eval_df.loc[eval_df['pos_model'] > threshold, 'neg'])
TN = np.sum(eval_df.loc[eval_df['neg_model'] < threshold, 'neg'])
FN = np.sum(eval_df.loc[eval_df['neg_model'] < threshold, 'pos'])  
print("*********** Results from Dhaval's code ***********")
print("TP: ", TP)
print("TN: ", TN)
print("FN: ", FN)
print("FP: ", FP)
print('Recall:', str(TP/(TP+FN)),  'Precision', str(TP/(TP+FP)))

*********** Results from Dhaval's code ***********
TP:  94
TN:  9
FN:  94
FP:  9
Recall: 0.5 Precision 0.912621359223301


In [35]:
# TP: tout ceux qui sont predits positifs et qui le sont
# Correct
np.sum(eval_df.loc[eval_df['pos_model'] > threshold, 'pos'])

94

In [40]:
# TN: tout ceux qui sont predits negatifs et qui le sont. 
# Erreur: relation de supériorité dans le mauvais sens. Proba d'etre 0 doit etre > au threshold pour predire 0
np.sum(eval_df.loc[eval_df['neg_model'] > threshold, 'neg'])

97

In [41]:
# FP: tout ceux qui sont predits positifs et qui ne le sont pas
# Correct
np.sum(eval_df.loc[eval_df['pos_model'] > threshold, 'neg'])

9

In [42]:
# FN: tout ceux qui sont predits negatifs et qui ne le sont pas
# Meme erreur que plus haut, relation de superiorite dans le mauvais sens
np.sum(eval_df.loc[eval_df['neg_model'] > threshold, 'pos'])

20